## WMP Maximo compactor analysis

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap

In [2]:
#set to display full column
pd.set_option('display.max_columns', None)

DATA CLEANING/CATEGORIZATION PROCESS (DEPRECATED)

1. Check data type for each column and make necessary conversions (i.e. datetime)
2. Split LOCATION string based on period into new columns
3. Move GND01 entries to ROOM column
4. Concatenate problem and failure columns
5. Export unique problem+failure entries
6. Manually classify problemandfailure codes based on observation and count of entries into:
    1. BULK
    2. INTCOMPACTOR
    3. EXTCOMPACTOR
    4. PESTS
    5. PAINT
    6. PLUMBING
    7. DOOR
    8. ELEC
    9. LIGHT
    10. OTHER
7. Add categorizations back into main dataframe
8. Export miscategorized compactor entries for manual reclassification (403 interior compactor entries located in 'GROUNDS')
9. Join (overwrite using .update) manually corrected miscategorized compactor entries
10. Write cleaned csv file for analysis

In [ ]:
# #load csv
# comp_data_raw = pd.read_csv("comp_raw.csv")

In [ ]:
##1A. Checking data types before processing to avoid errors
# print(comp_data_raw.dtypes) 

In [ ]:
# #1B. Convert date columns to datetime
# comp_data_raw["REPORTDATE"] = pd.to_datetime(comp_data_raw["REPORTDATE"])
# comp_data_raw["ZZCREATEDATE"] = pd.to_datetime(comp_data_raw["ZZCREATEDATE"])
# comp_data_raw["SCHEDSTART"] = pd.to_datetime(comp_data_raw["SCHEDSTART"],errors='coerce')
# comp_data_raw["TARGSTARTDATE"] = pd.to_datetime(comp_data_raw["TARGSTARTDATE"], errors='coerce')
# comp_data_raw["ACTSTART"] = pd.to_datetime(comp_data_raw["ACTSTART"])
# comp_data_raw["ACTFINISH"] = pd.to_datetime(comp_data_raw["ACTFINISH"])
# comp_data_raw["STATUSDATE"] = pd.to_datetime(comp_data_raw["STATUSDATE"])

In [ ]:
##1C. Save data types to variable to check later if you're bugging out
##Deprecated
# data_types = comp_data_raw.dtypes

In [ ]:
# ## 2. Split "LOCATION"
# # KEY: 0 - TDS, 1 - BUILDING, 2 - STAIRHALL, 3 - ROOM
# # new data frame with split value columns 
# split_loc = comp_data_raw["LOCATION"].str.split(".", expand = True) 
# # assign new columns based on split
# comp_data_raw["TDS"]= split_loc[0] 
# comp_data_raw["BUILDING"]= split_loc[1]
# comp_data_raw["STAIRHALL"]= split_loc[2] 
# comp_data_raw["ROOM"]= split_loc[3] 

In [ ]:
# #3A. Copy GND01 and erase GND01 from BUILDING column
# comp_data_raw["ROOM"] = np.where(comp_data_raw.BUILDING.str.contains("GND01")== True, "GND01", comp_data_raw["ROOM"])
# comp_data_raw["BUILDING"] = np.where(comp_data_raw.BUILDING.str.contains('GND01') == True, "", comp_data_raw["BUILDING"])

In [ ]:
# #3B. Copy GND01 to ROOM column and erase GND01 from STAIRHALL column
# comp_data_raw["ROOM"] = np.where(comp_data_raw.STAIRHALL.str.contains("GND01")== True, "GND01", comp_data_raw["ROOM"])
# comp_data_raw["STAIRHALL"] = np.where(comp_data_raw.STAIRHALL.str.contains('GND01') == True, "", comp_data_raw["STAIRHALL"])

In [ ]:
# #4. Concatenate problem and failure code
# comp_data_raw['PROBLEMANDFAILURE'] = comp_data_raw['PROBLEMCODE'] + comp_data_raw['FAILURECODE']

In [ ]:
##5A. Create new df to output problem and failure codes
##DEPRECATED
#new_df = comp_data_raw.PROBLEMANDFAILURE.value_counts().rename_axis("unique_values").reset_index(name="counts")

In [ ]:
##5B. Export to csv 
##DEPRECATED
#new_df.to_csv(r'datadict.csv', index = False)

In [ ]:
## 6. Explanation of manual classification [NOT COMPLETED]:
## 1. BULK -- 
## 2. INTCOMPACTOR -- 
## 3. EXTCOMPACTOR --
## 4. PESTS --
## 5. PAINT --
## 6. PLUMBING --
## 7. DOOR --
## 8. ELEC --
## 9. LIGHT --
## 10. OTHER --

In [ ]:
# # 7A. Import manual work order categorizations
# work_ord_cat = pd.read_csv("workorder_data_dictionary.csv")

In [ ]:
# # 7B. Create blank dictionary w/ keys
# cat_dict = {"BULK":[],"INTCOMPACTOR":[],"EXTCOMPACTOR":[],"PESTS":[],"PAINT":[], "PLUMBING":[],"DOOR":[],"ELEC":[],"LIGHT":[],"OTHER":[]}

In [ ]:
# # 7C. Iterate over rows and add unique codes to dictionary based on key value
# for i in range(len(work_ord_cat)): 
#     cat_dict[work_ord_cat.loc[i, "CATEGORY"]].append(work_ord_cat.loc[i, "PROBLEMCODE_FAILURECODE_unique"]) 

In [ ]:
# # 7D. Create blank category column in raw data df
# comp_data_raw["CATEGORY"] = ""

In [ ]:
# # 7E. Test for equivalency PROBLEMANDFAILURE column == category dictionary and add categorization to empty "CATEGORY" column
# for key, value in cat_dict.items():
#     for item in value:
#         comp_data_raw["CATEGORY"] = np.where(comp_data_raw.PROBLEMANDFAILURE.str.contains(item, regex = False) == True, key, comp_data_raw["CATEGORY"])


In [ ]:
## 7F. Testing For Success of categorization process TM
## DEPRECATED
#np.where(comp_data_raw["CATEGORY"] == '')

In [ ]:
# ## 7G. Fix mistakes :/
# #print(comp_data_raw.loc[15546,'PROBLEMANDFAILURE'])
# #print(comp_data_raw.loc[23014,'PROBLEMANDFAILURE'])
# comp_data_raw.loc[15546,'CATEGORY'] = 'PAINT'
# comp_data_raw.loc[23014,'CATEGORY'] = 'OTHER'

In [ ]:
## 8A. Export interior compactors where CATEGORY == INTCOMPACTOR & ROOM == GND01
##DEPRECATED
#miscat_comp = comp_data_raw[(comp_data_raw.CATEGORY == "INTCOMPACTOR") & (comp_data_raw.ROOM == "GND01")]
#miscat_comp.to_csv(r'miscat_comp.csv', index = False)

In [ ]:
#8B. Manually correct 403 miscategorized entries. File is saved as miscat_comp.csv. Explanation of process:
#1. If basement, room, or interior compactor is entered anywhere, leave as INTCOMPACTOR and change "ROOM" column to "Unknown - Originally entered as 'GND01'"
#2. Else, change "CATEGORY" to 'EXTCOMPACTOR'
#3. Cross-check with Dev Data_ Use this one.xlsx spreadsheet to ensure exterior compactors exist at development
#4. If 'NOISYCOMPACTOR' is in the problem or failurecode, examine other columns to ensure it is actually an external compactor

In [ ]:
# #9A. Join re-categorized data to comp_data_raw
# recat_comp = pd.read_csv("miscat_comp.csv")
# comp_data_raw.update(recat_comp)

In [ ]:
# #9B. Reset date columns to datetime
# comp_data_raw["REPORTDATE"] = pd.to_datetime(comp_data_raw["REPORTDATE"])
# comp_data_raw["ZZCREATEDATE"] = pd.to_datetime(comp_data_raw["ZZCREATEDATE"])
# comp_data_raw["SCHEDSTART"] = pd.to_datetime(comp_data_raw["SCHEDSTART"],errors='coerce')
# comp_data_raw["TARGSTARTDATE"] = pd.to_datetime(comp_data_raw["TARGSTARTDATE"], errors='coerce')
# comp_data_raw["ACTSTART"] = pd.to_datetime(comp_data_raw["ACTSTART"])
# comp_data_raw["ACTFINISH"] = pd.to_datetime(comp_data_raw["ACTFINISH"])
# comp_data_raw["STATUSDATE"] = pd.to_datetime(comp_data_raw["STATUSDATE"])

In [ ]:
# #10. Write cleaned file
# comp_data_raw.to_csv(r'comp_clnd.csv',index = False)

MAXIMO COMPACTOR CATEGORY SUMMARY STATISTICS PROCESS

Setup
1. Import 'comp_clnd.csv', set datetime, save datatypes
2. Group comp_clnd on consolidation, tds, and category to get the sums of each category for each development - tds_cat_grp
3. Pivot table the groupby tds_cat_grp to get columns for each type of work order category
4. Create list of categories for plotting

Consolidation Summary Stats Process
1. Groupby consolidation
2. Set up seaborn settings
3. For loop through df and export bar charts of category counts

Development Summary Stats Process
1. Create list of unique consolidations tds numbers
2. For loop through unique list returning dataframes == unique value
    3. For loop through each row for each development
        4. Drop columns with just one or less category
        
Room Location Summary Stats Process
1. 

In [3]:
#1A. Import comp_clnd.csv
comp = pd.read_csv("comp_clnd.csv")

#1B. Change to datetime and change TDS nums to int32
comp["REPORTDATE"] = pd.to_datetime(comp["REPORTDATE"])
comp["ZZCREATEDATE"] = pd.to_datetime(comp["ZZCREATEDATE"])
comp["SCHEDSTART"] = pd.to_datetime(comp["SCHEDSTART"],errors='coerce')
comp["TARGSTARTDATE"] = pd.to_datetime(comp["TARGSTARTDATE"], errors='coerce')
comp["ACTSTART"] = pd.to_datetime(comp["ACTSTART"])
comp["ACTFINISH"] = pd.to_datetime(comp["ACTFINISH"])
comp["STATUSDATE"] = pd.to_datetime(comp["STATUSDATE"])
comp.CONSOLIDATED_TDS_NUM = comp.CONSOLIDATED_TDS_NUM.astype('int32')
comp.TDS_NUM = comp.TDS_NUM.astype('int32')
comp.TDS = comp.TDS.astype('int32')
#1C. Save data types to variable to check later if you're bugging out
data_types = comp.dtypes

C:\Users\Justin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#2. Group comp_clnd by consolidation, tds, num for both tds and consolidations 
#   to get the sums of each kind of category for each TDS
tds_cat_grp = comp.groupby(['CONSOLIDATED_TDS_NUM', 'CONSOLIDATED_NAME','TDS_NUM','DEVELOPMENT_NAME','CATEGORY']).size().reset_index(name='Freq')

In [5]:
#3. Pivot tds_cat_grp to create columns for each type of compactor category
tds_cats_cnt = tds_cat_grp.pivot_table(index=['CONSOLIDATED_TDS_NUM', 'CONSOLIDATED_NAME','TDS_NUM','DEVELOPMENT_NAME'], columns='CATEGORY', values='Freq').reset_index().rename_axis(None, axis=1)
#fill na with 0 for graphs
tds_cats_cnt = tds_cats_cnt.fillna(0)
#change all floats to int
tds_cats_cnt[tds_cats_cnt.columns[4:14]] = tds_cats_cnt[tds_cats_cnt.columns[4:14]].astype(int)
#Reset column order
tds_cats_cnt = tds_cats_cnt[["CONSOLIDATED_NAME", "CONSOLIDATED_TDS_NUM", "DEVELOPMENT_NAME", "TDS_NUM", "INTCOMPACTOR","EXTCOMPACTOR", "PESTS", "PLUMBING", "PAINT", "BULK", "DOOR", "ELEC", "LIGHT","OTHER"]]
#Rename columns for legibility
tds_cats_cnt = tds_cats_cnt.rename(columns={'INTCOMPACTOR': 'Interior Compactor', 'EXTCOMPACTOR': 'Exterior Compactor', 'PESTS': 'Pests', 'PLUMBING':'Plumbing','ELEC': 'Electricity', 'LIGHT':'Lights', 'PAINT':'Painting', 'DOOR':'Door', 'BULK':'Bulk', 'OTHER': 'Other'})
#taking categories and turning into a list for graphs
cat_lst = list(tds_cats_cnt.columns[4:14])

Consolidation Summary Stats

In [ ]:
# #1. Groupby consolidation and sum category columns
# consol_cats_cnt = tds_cats_cnt.groupby(['CONSOLIDATED_TDS_NUM', 'CONSOLIDATED_NAME']).sum().reset_index().drop(columns=["TDS_NUM"])

In [ ]:
# #2. Setup for plotting - set style and create custom color palette

# #set style
# sns.set(style="whitegrid")

# #custom color pallette (connected commmunities color palette)
# con_colors = ['#587EBF', '#548F8F', '#FD765B', '#C86FB4','#8FC7A1', '#AEE9FA']
# #setting palette
# sns.set_palette(sns.color_palette(con_colors))


In [ ]:
# #3. Loop through every row and create a graph based on consolidation information
# for ind in range(len(cons_cat)):
#     #zip data to axes
#     ax = sns.barplot(x=cat_lst,y=consol_cats_cnt.iloc[ind,2:12])
#     #set axis titles
#     ax.set(xlabel="Work Type", ylabel = "# of Work Orders")
#     #set tick label rotation to display full label
#     ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
#     #set titles
#     ax.set_title('\n'.join(wrap(consol_cats_cnt.iloc[ind,1]+" Consolidation Work Orders in Compactor Locations 1/1/2019-7/1/2020")))
#     #honestly not sure this is changing anything but hell, a tighter graph would be great
#     plt.tight_layout()
#     #saving as svg
#     plt.savefig('Consolidation_BarCharts/svg/'+str(consol_cats_cnt.iloc[ind,0])+'_'+consol_cats_cnt.iloc[ind,1]+'_WorkOrder_Category_BarChart.svg')
#     #saving as png
#     plt.savefig('Consolidation_BarCharts/png/'+str(consol_cats_cnt.iloc[ind,0])+'_'+consol_cats_cnt.iloc[ind,1]+'_WorkOrder_Category_BarChart.png', dpi=300)
#     #closing axes, and figure etc
#     plt.cla()
#     plt.clf()

Development Summary Stats

In [7]:
#1. Setup for plotting - set style and create custom color palette

#set style
sns.set(style="whitegrid")

#custom color pallette (connected commmunities color palette)
con_colors = ['#587EBF', '#548F8F', '#FD765B', '#C86FB4','#8FC7A1', '#AEE9FA']
#setting palette
sns.set_palette(sns.color_palette(con_colors))


In [9]:
# #2. Iterate through and export graphs
# #iterating through unique consolidation numbers
# for consol_num in tds_cats_cnt.CONSOLIDATED_TDS_NUM.unique().tolist():
#     #testing if the consolidation only has one development in it
#     if len(tds_cats_cnt.loc[tds_cats_cnt['CONSOLIDATED_TDS_NUM'] == consol_num]) == 1:
#         pass
#     else:
#         #creating df of the consolidation
#         consol_df = tds_cats_cnt.loc[tds_cats_cnt['CONSOLIDATED_TDS_NUM'] == consol_num].reset_index(drop=True)
#         #iterating through every row
#         for ind in range(len(consol_df)):
#             #print(str(consol_df.iloc[ind,2]))
#             #create dataframe from row
#             tds_df = pd.DataFrame(consol_df.loc[ind]).transpose()
#             #jettison zero values from row
#             tds_df = tds_df.loc[:, (tds_df !=0).any(axis=0)]
#             #test if len of dataframe is only one category
#             if len(tds_df.iloc[0,4:]) <= 1:
#                 pass
#             else:
#             #create list of categories to zip
#                 cats = tds_df.iloc[0,4:].index.tolist()
#             #zip data to axes
#                 ax = sns.barplot(x=cats,y=tds_df.iloc[0,4:])
#             #set axis titles
#                 ax.set(xlabel="Work Type", ylabel = "# of Work Orders")
#             #set tick label rotation to display full label
#                 ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
#             #set titles
#                 ax.set_title('\n'.join(wrap(consol_df.iloc[ind,2].title()+" Development Work Orders in Compactor Locations 1/1/2019-7/1/2020")))
#             #honestly not sure this is changing anything but hell, a tighter graph would be great
#                 plt.tight_layout()
#             #saving as svg
#                 plt.savefig('Development_BarCharts/svg/'+str(consol_df.iloc[ind,3])+'_'+str(consol_df.iloc[ind,2])+'_WorkOrder_Category_BarChart.svg')
#             #saving as png
#                 plt.savefig('Development_BarCharts/png/'+str(consol_df.iloc[ind,3])+'_'+str(consol_df.iloc[ind,2])+'_WorkOrder_Category_BarChart.png', dpi=300)
#             #closing axes, and figure etc
#                 plt.cla()
#                 plt.clf()


Room Location Summary Stats

In [ ]:
#1. Setup for plotting - set style and create custom color palette

#set style
sns.set(style="whitegrid")

#custom color pallette (connected commmunities color palette)
con_colors = ['#587EBF', '#548F8F', '#FD765B', '#C86FB4','#8FC7A1', '#AEE9FA']
#setting palette
sns.set_palette(sns.color_palette(con_colors))

In [20]:
#comp
tds_rm_grp = comp.groupby(['CONSOLIDATED_TDS_NUM', 'CONSOLIDATED_NAME','TDS_NUM','DEVELOPMENT_NAME','CATEGORY','BUILDING', 'ROOM']).size().reset_index(name='Freq')

In [23]:
tds_rm_grp[tds_rm_grp["Freq"]>5]

,CONSOLIDATED_TDS_NUM,CONSOLIDATED_NAME,TDS_NUM,DEVELOPMENT_NAME,CATEGORY,BUILDING,ROOM,Freq
19,3,Harlem River,3,HARLEM RIVER,INTCOMPACTOR,02,F01,8
38,3,Harlem River,3,HARLEM RIVER,PAINT,01,F01,8
39,3,Harlem River,3,HARLEM RIVER,PAINT,02,F01,7
40,3,Harlem River,3,HARLEM RIVER,PAINT,03,F01,6
41,3,Harlem River,3,HARLEM RIVER,PAINT,04,F01,8
...,...,...,...,...,...,...,...,...
10986,505,Queensbridge North,505,QUEENSBRIDGE NORTH,PESTS,21,COM01,6
10991,505,Queensbridge North,505,QUEENSBRIDGE NORTH,PESTS,22,COM01,6
11028,514,Whitman,514,WHITMAN,ELEC,38,F01,7
11030,514,Whitman,514,WHITMAN,EXTCOMPACTOR,10,GND01,14


In [ ]:
#for each development, if a building's room  